# Introducing Convolutional Layer with a Twist

I'm excited to introduce `conv_twist`, a replacement of (and an improvement on) the good old **convolutional layer** widely used in Deep Learning models (rightfully referred to as ConvNets or CNN) in Computer Vision. Famously introduced by Yann LeCun some 30 years ago into image classification, it became the source of the Deep Learning/Artificial Intelligence revolution with AlexNet in 2012. Rapid improvements on the architecture followed, most notably the ResNet of 2015. Recently attention has somewhat shifted away from image classification, but convolutional layers are still the bread and butter of any Computer Vision models. What more can be said about convolutional layers, one might ask? The answer comes from a little bit of mathematics.

Long story short, here is one implementation of `conv_twist` in PyTorch, and you can easily swap out the 3x3 `Conv2d` in your model and plug this in, and train from scratch to see if it gives any improvement. (Report on results are welcome.)

In [29]:
import torch
import torch.nn as nn
import torchvision

class conv_twist(nn.Module):  # replacing 3x3 Conv2d
    def __init__(self, ni, nf, init_max=1.5, stride=1):
        super(conv_twist, self).__init__()
        self.conv = nn.Conv2d(ni, nf, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv_x = nn.Conv2d(ni, nf, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv_y = nn.Conv2d(ni, nf, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv_x.weight.data = (self.conv_x.weight - self.conv_x.weight.flip(2).flip(3)) / 2  # make conv_x a "first-order operator" by symmetrizing it
        self.conv_y.weight.data = self.conv_x.weight.transpose(2,3).flip(3)                      # make conv_y a 90 degree rotation of convx
        self.center_x = nn.Parameter(torch.Tensor(nf), requires_grad=True)
        self.center_y = nn.Parameter(torch.Tensor(nf), requires_grad=True)
        self.center_x.data.uniform_(-init_max, init_max)
        self.center_y.data.uniform_(-init_max, init_max)

    def forward(self, x):
        self.conv_x.weight.data = (self.conv_x.weight - self.conv_x.weight.flip(2).flip(3)) / 2  
        self.conv_y.weight.data = (self.conv_y.weight - self.conv_y.weight.flip(2).flip(3)) / 2
        x1 = self.conv(x)
        _, c, h, w = x1.size()
        XX = torch.from_numpy(np.indices((1,h,w))[2]*2/w).type(x.dtype).to(x.device) - self.center_x.view(-1,1,1)
        YY = torch.from_numpy(np.indices((1,h,w))[1]*2/h).type(x.dtype).to(x.device) - self.center_y.view(-1,1,1)
        return x1 + (XX * self.conv_x(x) + YY * self.conv_y(x))

Let's take a look at the weights in such a `conv_twist` model:

In [30]:
model = conv_twist(1,1)
for name, para in model.named_parameters():
    print(name, para)

center_x Parameter containing:
tensor([0.2157], requires_grad=True)
center_y Parameter containing:
tensor([0.9144], requires_grad=True)
conv.weight Parameter containing:
tensor([[[[ 0.0164,  0.0767, -0.2709],
          [-0.3086, -0.0439,  0.1440],
          [-0.2906,  0.0374,  0.2853]]]], requires_grad=True)
conv_x.weight Parameter containing:
tensor([[[[ 0.2468, -0.2211, -0.2301],
          [-0.0661,  0.0000,  0.0661],
          [ 0.2301,  0.2211, -0.2468]]]], requires_grad=True)
conv_y.weight Parameter containing:
tensor([[[[ 0.2301, -0.0661,  0.2468],
          [ 0.2211,  0.0000, -0.2211],
          [-0.2468,  0.0661, -0.2301]]]], requires_grad=True)


If you take a look at the `conv_x` weights, you'd notice that each 3x3 kernel is symmetric (the numbers on the opposite ends are identical except for the signs, with the middle one always 0). That's the effect of "symmetrizing" on `conv_x`, done at each forward pass. You can also check that the `conv_x` and `conv_y` weights are initialized to be identical but off by a 90 degree rotation.

Why do I choose to initialize the weights this way? Well, I'll try to explain all this later. For now it's important to note that `conv_twist` is a lot bigger than the standard `Conv2d` layer, but not as much as it appears to be. This particular implementation, if I had done it properly, has about twice as many trainable weights as a single `Conv2d` layer.

## What are convolutions?

The classic storyline in Neural Networks is that the convolution operator captures the spatial relation of the pixels, so is particularly suited for image-related learning task, and has much fewer weights than a generic linear map (fully connected layer). And over the years people learned that we don't need kernels that are larger than 3x3, and to go deeper (i.e., many layers) instead, hence *deep* learning.

What is perhaps not well-known is that different 3x3 kernels have rather intuitive meanings, in terms of what it does to the image *overall*. For example, the Gaussian kernel in image processing "blurs" the image. We can do a little experiment to see:

In [33]:
G = torch.Tensor([[1,2,1],[2,4,2],[1,2,1]])/16
conv = nn.Conv2d(1,1,kernel_size=3, bias=False)
conv.weight.data = G
for name, param in conv.named_parameters():
    print(name, param)
    
# take a grayscale image, and feed it into the conv model. Display the result as an image side-by-side the original.

weight Parameter containing:
tensor([[0.0625, 0.1250, 0.0625],
        [0.1250, 0.2500, 0.1250],
        [0.0625, 0.1250, 0.0625]], requires_grad=True)


To illustrate the effect for other 3x3 kernels, it seems best to choose the kernel close to the "identity", and to apply it many times. Try the following:

In [31]:
A = torch.Tensor([[-1,0,1],[-2,0,2],[-1,0,1]])
B = torch.Tensor([[1,2,1],[0,0,0],[-1,-2,-1]])
I = torch.Tensor([[0,0,0],[0,1,0],[0,0,0]])
K = I + 0.01 * A

## What does `conv_twist` do?

In addition to a normal Conv2d layer, the `conv_twist` is feeding the input to two other 3x3 Conv2d layer. To see the effect, let's turn off the `conv` weights

In [37]:
model.conv.data = 0
model.center_x.data = torch.Tensor([0])
model.center_y.data = torch.Tensor([0])
model.conv_x.data = I + 0.01 * A
model.conv_y.data = I + 0.01 * B